In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import os
from typing import List, Tuple, Dict
import pandas as pd
import numpy as np

import osmnx as ox
import networkx as nx
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from shapely.geometry import Point

from dotenv import load_dotenv

load_dotenv('../.env')
# print("PYTHONPATH:", os.environ.get('PYTHONPATH'))

import geo_utils
df = pd.read_csv('../data/lokationen-geodaten.csv')

In [46]:
df = df.rename(columns={' "lat"': 'lat', ' "lng"': 'long', ' "Ort"': 'Ort', ' "ID"': 'ID'})
df['lat'] = df['lat'].str.strip().str.replace('"', '', regex=False).replace('', np.nan).astype(float)
df['long'] = df['long'].str.strip().str.replace('"', '', regex=False).replace('', np.nan).astype(float)
df.head(5)
# geo_utils.get_locations_within_range(center_long: float, center_lat: float, n: float, df: pd.DataFrame)

,Bezeichnung,long,lat,Ort,ID
0,Besitos Münster,7.632360,51.962471,"""Münster""","""20b68cc5-c5c1-41ee-8518-b20973171c25"""
1,Bistro GreenBee,9.079920,52.187170,"""Rinteln""","""G2z0yxF76474q2z1EsRxj"""
2,Bodega Beach Club,9.080265,52.190996,"""Rinteln""","""rJ3eg2BbsmMS427SkpnH"""
3,Bodega Marktplatz 9,9.081009,52.187531,"""Rinteln""","""qaZycWKmyxiBbyE8XvLd"""
4,Bodega Verden,9.231492,52.921643,"""Verden""","""bk5y74SLJ7CdWpIXdxJzN"""


In [47]:
df = df.dropna(subset=['lat', 'long']).reset_index(drop=True)

In [48]:
searchers = [
    (53.0, 9.17, "Test Verden"),
    (52.425435999, 9.424194999, "Test Wunstorf"),
    (52.32010, 9.20780, "Test Stadthagen"),
    (52.2116, 8.8087, "Test Bad Oeynhausen"),
    (52.02830, 7.11400, "Test Münster"),
    (52.18698, 9.07984, 'Test Rinteln')
]

In [49]:
# Show total map
geo_utils.plot_points_on_map(df, searchers=searchers)

In [69]:
searcher = searchers[1]
near_by_search_result = geo_utils.get_locations_within_range(lat=searcher[0], long=searcher[1], n=300, df=df)
geo_utils.plot_points_on_map(near_by_search_result, searchers=[searcher])

In [5]:
class DeliveryRouter:
    def __init__(self, city: str = "Berlin, Germany"):
        """Initialize the router with a specific city's road network."""
        # Initialize Nominatim geocoder
        self.geocoder = Nominatim(user_agent="delivery_router_rinteln", timeout=6)
        self.city_boundary = ox.geocode_to_gdf(city)

        # Download and create the road network for the specified city
        self.G = ox.graph_from_place(city, network_type="drive", simplify=False)
        
        # Convert to projected graph for accurate distance calculations; UTM zone 32N
        self.G_proj = ox.project_graph(self.G, to_crs="EPSG:25832")
        
        # Store restaurants data
        self.restaurants = {}
        self.load_all_restaurants()
        
    def load_all_restaurants(self):
        """Load all restaurants from OpenStreetMap within the city boundary."""
        # Define tags for restaurants and similar establishments
        tags = {
            'amenity': ['restaurant', 'cafe', 'fast_food', 'bar', 'pub'],
            'cuisine': True  # This will get all cuisine types
        }
        
        # Get all POIs with these tags within the city boundary
        pois = ox.features_from_place("Rinteln, Germany", tags=tags)
        
        # Process each POI
        for idx, poi in pois.iterrows():
            name = poi.get('name', 'Unknown Restaurant')
            if name != 'Unknown Restaurant':  # Only add places with names
                try:
                    # Get coordinates
                    if isinstance(poi.geometry, Point):
                        lat, lon = poi.geometry.y, poi.geometry.x
                    else:
                        # For polygons, use centroid
                        lat, lon = poi.geometry.centroid.y, poi.geometry.centroid.x
                    
                    # Get address components if available
                    street = poi.get('addr:street', '')
                    housenumber = poi.get('addr:housenumber', '')
                    address = f"{street} {housenumber}, Rinteln" if street and housenumber else "Rinteln"
                    
                    # Store restaurant data
                    self.restaurants[name] = {
                        "address": address,
                        "lat": lat,
                        "lon": lon,
                        "node": ox.nearest_nodes(self.G, lon, lat),
                        "type": poi.get('amenity', 'unknown'),
                        "cuisine": poi.get('cuisine', 'unknown')
                    }
                    print(f"Added: {name} ({address})")
                except Exception as e:
                    print(f"Error adding {name}: {e}")

    def add_restaurant(self, name: str, address: str) -> bool:
        """Add a restaurant to the system with its location."""
        try:
            location = self.geocoder.geocode(f"{name}, {address}, Germany")
            if location:
                self.restaurants[name] = {
                    "address": address,
                    "lat": location.latitude,
                    "lon": location.longitude,
                    "node": ox.nearest_nodes(self.G, location.longitude, location.latitude)
                }
                return True
            return False
        except Exception as e:
            print(f"Error adding restaurant: {e}")
            return False
    
    def calculate_route_advanced(self, start_coords: Tuple[float, float], 
                        end_coords: Tuple[float, float]) -> List[Tuple[float, float]]:
        """Calculate a detailed route between two points."""
        start_node = ox.nearest_nodes(self.G, start_coords[1], start_coords[0])
        end_node = ox.nearest_nodes(self.G, end_coords[1], end_coords[0])
        
        # Find the shortest path by nodes
        path = nx.shortest_path(self.G, start_node, end_node, weight="length")
        
        # Retrieve the detailed geometry of the route
        route_coords = []
        for u, v in zip(path[:-1], path[1:]):  # Iterate over edges in the path
            edge_data = self.G[u][v][0]  # Get the edge data
            if "geometry" in edge_data:  # Use geometry if available
                route_coords.extend([(point[1], point[0]) for point in edge_data["geometry"].coords])
            else:  # Fall back to straight line if no geometry is present
                route_coords.extend([(self.G.nodes[u]['y'], self.G.nodes[u]['x']),
                                    (self.G.nodes[v]['y'], self.G.nodes[v]['x'])])
        
        return route_coords

    
    def visualize_map(self, driver_position: Tuple[float, float] = None, 
                     route: List[Tuple[float, float]] = None) -> folium.Map:
        """Create a visualization of restaurants, driver, and route."""
        # Create base map centered on Rinteln
        center_location = self.geocoder.geocode("Rinteln, Germany")
        m = folium.Map(location=[center_location.latitude, center_location.longitude], 
                      zoom_start=14)
        
        # Add restaurants with custom icons
        for name, data in self.restaurants.items():
            folium.Marker(
                [data["lat"], data["lon"]],
                popup=f"{name}\n{data['address']}",
                icon=folium.Icon(color='red', icon='cutlery')
            ).add_to(m)
        
        # Add driver position if provided
        if driver_position:
            folium.Marker(
                driver_position,
                popup="Driver Location",
                icon=folium.Icon(color='green', icon='user')
            ).add_to(m)
        
        # Add delivery route if provided
        if route:
            folium.PolyLine(
                route,
                weight=3,
                color='blue',
                opacity=0.8
            ).add_to(m)
        
        return m

    def get_estimated_delivery_time(self, route: List[Tuple[float, float]], 
                                  avg_speed_kmh: float = 30) -> float:
        """Calculate estimated delivery time in minutes for a route."""
        total_distance = 0
        for i in range(len(route) - 1):
            total_distance += geodesic(route[i], route[i + 1]).kilometers
        
        # Convert to hours and then to minutes
        return (total_distance / avg_speed_kmh) * 60


In [19]:
# Initialize router - it will automatically load all restaurants
router = DeliveryRouter("Rinteln, Germany")

# Add delivery address
delivery_address = "6, Bachweg, Rinteln, 31737"
try:
    delivery_location = router.geocoder.geocode(delivery_address)
    driver_position = (delivery_location.latitude, delivery_location.longitude)
    print(f"Delivery location found: {delivery_location.latitude}, {delivery_location.longitude}")
except Exception as e:
    print(f"Error geocoding delivery address: {e}")

# Print all found restaurants
print("\nAll restaurants found in Rinteln:")
# for name, data in router.restaurants.items():
    # print(f"{name}: {data['address']} ({data['type']}, {data['cuisine']})")
print(f"len(router.restaurants.items()) {len(router.restaurants.items())}")
# Create and save map
map_view = router.visualize_map(
    driver_position=driver_position
)

map_view.save('rinteln_all_restaurants_map.html')

Added: Italia (Pferdemarkt 16, Rinteln)
Added: Ipeks Döneria (Konrad-Adenauer-Straße 34, Rinteln)
Added: nan (nan nan, Rinteln)
Added: La Taverna (nan nan, Rinteln)
Added: Zum Pfingsttor (Pfingsttorstraße 24, Rinteln)
Added: Burg-Gaststätte (nan nan, Rinteln)
Added: Da Fabio (Extertalstraße 41, Rinteln)
Added: Prasuhn's Hofcafé (Landstraße 13, Rinteln)
Added: Waldgasthaus Homberg (Kleine Heide 9, Rinteln)
Added: Eiscafé Rialto (Marktplatz 5, Rinteln)
Added: Klönklause (nan nan, Rinteln)
Added: Odysseus Grill (Brennerstraße 16, Rinteln)
Added: Asia Palast (Mühlenstraße 8, Rinteln)
Added: McDonald's (Konrad-Adenauer-Straße 44, Rinteln)
Added: Freizeit-Center Schubertweg (Schubertweg 44, Rinteln)
Added: Hotel Gasthaus Zur Linde (Hauptstraße 30, Rinteln)
Added: Shang-Hai (Stoevesandtstraße 4, Rinteln)
Added: Schildkröte (nan nan, Rinteln)
Added: Pizzeria Rino (Bahnhofsallee 12, Rinteln)
Added: Bodega Rinteln (nan nan, Rinteln)
Added: Kuddel's Grillstube (Bahnhofstraße 5A, Rinteln)
Added: A

In [7]:
router = DeliveryRouter(city="Rinteln, Germany")

# Add restaurants with full addresses and increased timeout
restaurants = {
    "Bodega": "31737 Rinteln",
    "Mykonos": "31737 Rinteln",
    "Bodega Beach Bar": "31737 Rinteln",
    "Kuddel's Grillstube": "31737 Rinteln",
    "Pizzeria Corallo": "31737 Rinteln",
    "Da Fabio": "31737 Rinteln"
}

# Add each restaurant and print the results
for name, address in restaurants.items():
    success = router.add_restaurant(name, address)
    print(f"Adding {name}: {'Success' if success else 'Failed'}")

# Add delivery address
delivery_address = "6, Bachweg, Rinteln, 31737"
try:
    delivery_location = router.geocoder.geocode(delivery_address)
    driver_position = (delivery_location.latitude, delivery_location.longitude)
    print(f"Delivery location found: {delivery_location.latitude}, {delivery_location.longitude}")
except Exception as e:
    print(f"Error geocoding delivery address: {e}")

Adding Bodega: Success
Adding Mykonos: Success
Adding Bodega Beach Bar: Success
Adding Kuddel's Grillstube: Success
Adding Pizzeria Corallo: Success
Adding Da Fabio: Success
Delivery location found: 52.2052102, 9.0932679


In [6]:
# beach_bar_data = router.restaurants["Bodega Beach Bar"]
# destination = (beach_bar_data["lat"], beach_bar_data["lon"])

# route = router.calculate_route_advanced(
#     start_coords=driver_position,
#     end_coords=destination
# )
# # route_map = ox.plot.plot_graph(router.G)
# # ox.plot.plot_graph_route(router.G, route, route_linewidth=6, node_size=0, bgcolor='k')
# print(f"route {route}")
# # Create and save map without route calculation for now
# map_view = router.visualize_map(
#     driver_position=driver_position,
#     route=route
# )
# map_view.save('rinteln_delivery_map.html')

# # Print all successfully geocoded locations
# print("\nStored restaurant locations:")
# for name, data in router.restaurants.items():
#     print(f"{name}: {data['lat']}, {data['lon']}")